# For Extracting Census Data from the Census API and the Census REST Server.

## Notebook to assist in the process of redistricting for Kent County. Right now the data is going to be based on the Census ACS Survery. It is still unclear if the Offical Census Data will be avaiable before the process needs to be completed.

In [ ]:
from dotenv import find_dotenv, load_dotenv
import os
from arcgis import GIS

load_dotenv(find_dotenv())

gis_user = os.getenv('ESRI_USERNAME')
gis_pass = os.getenv('ESRI_PASSWORD')
gis_url = os.getenv('PORTAL_URL')
agol_user = os.getenv('AGOL_USERNAME')
agol_pass = os.getenv('AGOL_PASSWORD')
agol_url = os.getenv('AGOL_SITE')

gisE = GIS(url=gis_url, username=gis_user, password=gis_pass)

gisA = GIS(url=agol_url, username=agol_user, password=agol_pass)

gis = GIS()

### Get Census Data for Kent County from ACS 5 Year Survey. Data is going to be at the block group level instead of the block level. 

In [ ]:
import pandas as pd
import numpy as np
import requests

census_key = os.getenv('CENSUS_KEY')

cvar = {
    'GEO_ID':'GEOID',
    'B01001_001E':'est_total',
    'B01001_001EA':'est_total_anno',
    'B01001_001M':'moe',
    'B01001_001MA':'moe_anno',
    'NAME':'name'
}

# Request parameters
payload = {
    'get' : ','.join(list(cvar.keys())),
    'for' : 'block group:*',
    'in' : ['state:10', 'county:001'],
    'key' : census_key
}

census_url = f'https://api.census.gov/data/2019/acs/acs5'

res = requests.get(census_url, params=payload)

if res.status_code == 200:
    # print(response.url)
    data = res.json()
    # print(data[0])

    
    c_df = pd.DataFrame(data[1:], columns=data[0])
    c_df.rename(columns=cvar, inplace=True)

In [ ]:
c_df.loc[:, 'GEOID'] = c_df.GEOID.str[9:]

c_df.drop(columns=['state', 'county', 'tract', 'block group'], inplace=True)

c_df.sort_values(by='GEOID', inplace=True, ignore_index=True)

census_dtypes = {
    'GEOID':'string',
    'name':'string',
    'est_total':'int32',
    'est_total_anno':'string',
    'moe':'int32',
    'moe_anno':'string'
}

c_df = c_df[list(census_dtypes.keys())].astype(census_dtypes)


In [ ]:
c_df.fillna('', inplace=True)
c_df.isna().sum()

### Get Census Block Group Geographic Data from Census REST Server. For this example, I am using ArcGIS API for Python, but you can use Open Source libraries like Geopandas.

In [ ]:
from pathlib import Path

cwd = Path.cwd()
gis = cwd / 'census.shp'

In [ ]:
from arcgis.features import FeatureLayer, FeatureLayerCollection

flc = FeatureLayerCollection('https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer')

expr = 'STATE = 10 AND COUNTY = 01'

flList = flc.layers

cbg = flList[0].query(out_sr=4326, where=expr, as_df=True)
print(cbg)
cbg.spatial.to_featureclass(f'{gis}', sanitize_columns=False)
# cbg_df = FeatureLayer(fl).query(out_sr= 4326, where=expr, out_fields='GEOID',as_df=True)

# cbg_df = FeatureLayer(fl).query(out_sr= 4326, where=expr, as_df=True)

# cbg_df.sort_values(by='GEOID', inplace=True, ignore_index=True)

# cbg_df.spatial.to_featureclass(f'{gis}', sanitize_columns=False)

# cbg_df.save(f'{gis.parent}', 'census.shp')
# print('Done')


In [12]:
cbg_df

,MTFCC,OID,GEOID,STATE,COUNTY,TRACT,BLKGRP,BASENAME,NAME,LSADC,FUNCSTAT,AREALAND,AREAWATER,CENTLAT,CENTLON,INTPTLAT,INTPTLON,OBJECTID,SHAPE
0,G5030,20859404539530,100010420001,10,001,042000,1,1,Block Group 1,BG,S,36919396,124382,+39.0865047,-075.6802580,+39.0933119,-075.6893622,66993,"{""rings"": [[[-75.74234799957848, 39.0801009999..."
1,G5030,20859404541272,100010417012,10,001,041701,2,2,Block Group 2,BG,S,5846791,0,+39.1024344,-075.5566006,+39.1024344,-075.5566006,66994,"{""rings"": [[[-75.58080600031265, 39.0944429996..."
2,G5030,20859404540083,100010414001,10,001,041400,1,1,Block Group 1,BG,S,6052970,0,+39.1462649,-075.5445781,+39.1462649,-075.5445781,66995,"{""rings"": [[[-75.56096200009078, 39.1586879999..."
3,G5030,20859404539943,100010433001,10,001,043300,1,1,Block Group 1,BG,S,4451359,102108,+39.1883488,-075.5524394,+39.1901603,-075.5513183,66996,"{""rings"": [[[-75.57117299990671, 39.1910680001..."
4,G5030,20859404540649,100010401001,10,001,040100,1,1,Block Group 1,BG,S,49660039,0,+39.2698848,-075.7023840,+39.2698848,-075.7023840,66998,"{""rings"": [[[-75.76001799955766, 39.2968229999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,G5030,20859404540511,100010430003,10,001,043000,3,3,Block Group 3,BG,S,14158636,18927,+38.9249920,-075.5550856,+38.9245258,-075.5554309,208162,"{""rings"": [[[-75.57230799970863, 38.8889220002..."
68,G5030,20859404540591,100010431001,10,001,043100,1,1,Block Group 1,BG,S,69929674,0,+38.9246882,-075.6746398,+38.9246882,-075.6746398,208163,"{""rings"": [[[-75.73292599995807, 38.9573919997..."
69,G5030,20859404540188,100010417021,10,001,041702,1,1,Block Group 1,BG,S,3459137,0,+39.1108318,-075.5154754,+39.1108318,-075.5154754,210326,"{""rings"": [[[-75.53499299969921, 39.1129759997..."
70,G5030,20859404540215,100010417022,10,001,041702,2,2,Block Group 2,BG,S,2554429,0,+39.1030795,-075.5332942,+39.1030795,-075.5332942,210327,"{""rings"": [[[-75.54594699967991, 39.0952159998..."


<b> Method using GeoPandas instead of ArcGIS Python API. </b>

In [ ]:
import geopandas as gpd

blocks_url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/8/query?'

blocks_params = {
    'where' : 'STATE = 10 and COUNTY = 001',
    'outFields' : 'GEOID',
    'f' : 'GeoJSON'
}

blocks = requests.get(blocks_url, blocks_params)

cbg_gdf = gpd.read_file(blocks.text)



cbg_gdf.sort_values(by='GEOID', inplace=True, ignore_index=True)
cbg_gdf

In [ ]:
import geopandas as gpd
import requests

blocks_url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/8/query?'

blocks_params = {
    'where' : 'STATE = 10 and COUNTY = 001',
    'outFields' : '*',
    'f' : 'JSON'
}

blocks = requests.get(blocks_url, blocks_params)

cbg_gdf = gpd.read_file(blocks.text)



cbg_gdf.sort_values(by='GEOID', inplace=True, ignore_index=True)
cbg_gdf

In [13]:
cbg_gdf.to_file(f'{gis.parent / "gdCensus.shp"}', index=False)

In [11]:
import fiona
fiona.supported_drivers

{'AeronavFAA': 'r',
 'ARCGEN': 'r',
 'BNA': 'rw',
 'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'r',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'rw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'rw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'GPSTrackMaker': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'rw',
 'OGR_PDS': 'r',
 'S57': 'r',
 'SEGY': 'r',
 'SUA': 'r',
 'TopoJSON': 'r'}

### Merge or join the Census API Data to Census Spatial Data 

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl

cbg_gdf.plot()

In [ ]:
print(cbg_df.columns)
print(cbg_gdf.columns)

In [ ]:
kc_gdf = cbg_gdf[['geometry']].join(c_df)
kc_df = cbg_df[['SHAPE']].join(c_df)

In [ ]:
# Define figure
fig, ax = plt.subplots(1,1)

# Plot dataframe to figure
div = make_axes_locatable(ax)
cax = div.append_axes("bottom", size='5%', pad=0.1)
kc_gdf.plot(column='est_total', legend=True, ax=ax, legend_kwds={'orientation':'horizontal'}, cax=cax, edgecolor='black', linewidth=0.4, cmap='BrBG')

# Adjust figure details
ax.set_title('Estimated Population Change from 2010-2014 to 2015-2019')
ax.axes.xaxis.set_visible(False); ax.axes.yaxis.set_visible(False)
ax.set_frame_on(False)
fig.set_figwidth(10); fig.set_figheight(10)

plt.show()

## Get Geographic Data from Census REST Server

In [ ]:
https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer